In [ ]:
# !pip install biopython

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   -------------- ------------------------- 1.0/2.8 MB 8.5 MB/s eta 0:00:01
   -------------------------- ------------- 1.8/2.8 MB 3.5 MB/s eta 0:00:01
   ----------------------------- ---------- 2.1/2.8 MB 3.8 MB/s eta 0:00:01
   ---------------------------------------- 2.8/2.8 MB 3.2 MB/s eta 0:00:00


In [1]:
import pandas as pd
import numpy as np
from Bio import SeqIO

In [2]:
def read_fasta(file_path, filter_header=None):
    sequences = ''
    for record in SeqIO.parse(file_path, "fasta"):
        if filter_header and filter_header not in record.id:
            continue
        sequences = record.seq
    return sequences

def read_centro_telo(file_path_centro, file_path_telo, file_path_cpg):
    centro = pd.read_csv(file_path_centro)
    centro.drop('#"bin"', axis=1, inplace=True)
    
    telo = pd.read_csv(file_path_telo)
    telo.drop('#bin', axis=1, inplace=True)
    telo = telo[telo['type'] == 'telomere']
    
    cpg = pd.read_csv(file_path_cpg)
    cpg.drop('#"bin"', axis=1, inplace=True)
    
    return centro, telo, cpg

def check_overlap(centro, telo, cpg):
    telo.sort_values('chromStart', inplace=True)
    # print("Starting Telomere")
    # print(f'({telo[['chromStart', 'chromEnd']].iloc[0, 0]},{telo[['chromStart', 'chromEnd']].iloc[0, 1]})')
    # print("Ending Telomere")
    # print(f'({telo[['chromStart', 'chromEnd']].iloc[1, 0]},{telo[['chromStart', 'chromEnd']].iloc[1, 1]})')
    
    start_telo = telo[['chromStart', 'chromEnd']].iloc[0, 1]
    end_telo = telo[['chromStart', 'chromEnd']].iloc[1, 0]

    if len(cpg[cpg['chromStart'] < start_telo]) > 0 or len(cpg[cpg['chromEnd'] > end_telo]) > 0:
        print("There is a CpG island overlapping with Telomeric regions.")
    else:
        print("There is NO overlapping CpG island with Telomeric regions.")
        
    centro.sort_values('chromStart', inplace=True)
    min_centro = centro[['chromStart','chromEnd']].iloc[0,0]
    max_centro = centro[['chromStart','chromEnd']].iloc[len(centro)-1, 1]

    if len(cpg[(cpg['chromStart'] > min_centro) & (cpg['chromEnd'] < max_centro)]) > 0:
        print("Do a manual inspection because there is a CpG island in between the Centromeric regions.")
    else:
        print("There is NO overlapping CpG island with Centromeric regions.")

In [3]:
seq = read_fasta('./GRCh38.p14.genome.fa', 'chr18')

In [4]:
centro18_df, telo18_df, cpg18_df = read_centro_telo('./chr18/centromeres.csv','./chr18/telomeres.csv', './chr18/cpg.csv')
centro19_df, telo19_df, cpg19_df = read_centro_telo('./chr19/centromeres.csv','./chr19/telomeres.csv', './chr19/cpg.csv')

In [5]:
check_overlap(centro18_df, telo18_df, cpg18_df)

There is NO overlapping CpG island with Telomeric regions.
There is NO overlapping CpG island with Centromeric regions.


In [6]:
check_overlap(centro19_df, telo19_df, cpg19_df)

There is NO overlapping CpG island with Telomeric regions.
There is NO overlapping CpG island with Centromeric regions.


In [33]:
def check_in_start_end(i, index, start_end, in_):
    if index < len(start_end):
        if not in_ and i >= start_end[index]:
            return True, index + 1
        if in_ and i >= start_end[index]:
            return False, index + 1
    return in_, index    

In [34]:
# check regions for ith base
in_cpg = False
cpg_index = 0

in_centro = False
centro_index = 0

in_telo = False
telo_index = 0

# check regions for (i+1)th base
in_cpg_next = False
cpg_index_next = 0

in_centro_next = False
centro_index_next = 0

in_telo_next = False
telo_index_next = 0

# flatten the regions
telo_start_end = telo18_df.sort_values('chromStart')[['chromStart', 'chromEnd']].to_numpy().flatten()
centro_start_end = centro18_df.sort_values('chromStart')[['chromStart', 'chromEnd']].to_numpy().flatten()
cpg_start_end = cpg18_df.sort_values('chromStart')[['chromStart', 'chromEnd']].to_numpy().flatten()

# Format: A-, G-, C-, T-, A+, G+, C+, T+
init_probs = np.zeros(8) # data structure for initial probabilities
emit_probs = np.concatenate([np.eye(4), np.eye(4)]) # data structure for emission probabilities
tran_probs = np.zeros((8,8)) # data structure for transition probabilities

# base to index
init_dict = {
    'A-': 0,
    'G-': 1,
    'C-': 2,
    'T-': 3,
    'A+': 4,
    'G+': 5,
    'C+': 6,
    'T+': 7,
}

# linearly traverse the sequence
for i in range(len(seq)):
    # check if in telomeric region
    in_telo, telo_index = check_in_start_end(i, telo_index, telo_start_end, in_telo)
    if in_telo:
        continue

    # check if in centromeric region
    in_centro, centro_index = check_in_start_end(i, centro_index, centro_start_end, in_centro)
    if in_centro:
        continue

    # check if in CpG island
    in_cpg, cpg_index = check_in_start_end(i, cpg_index, cpg_start_end, in_cpg)

    # avoid ambiguous bases N
    if seq[i] == 'N':
        continue

    # counting the frequency of each Markov States
    base_i = seq[i] + ('+' if in_cpg else '-')
    init_probs[init_dict[base_i]] += 1

    if i + 1 < len(seq):
        # check if next index in telomeric region
        in_telo_next, telo_index_next = check_in_start_end(i + 1, telo_index_next, telo_start_end, in_telo_next)
        if in_telo_next:
            continue

        # check if next base in centromeric region
        in_centro_next, centro_index_next = check_in_start_end(i + 1, centro_index_next, centro_start_end, in_centro_next)
        if in_centro_next:
            continue

        # check if next base in CpG island
        in_cpg_next, cpg_index_next = check_in_start_end(i + 1, cpg_index_next, cpg_start_end, in_cpg_next)

        # avoid ambiguous bases N
        if seq[i+1] == 'N':
            continue

        base_i_next = seq[i + 1] + ('+' if in_cpg_next else '-')
        tran_probs[init_dict[base_i_next], init_dict[base_i]] += 1

In [86]:
# sanity check for initial probabilities
(init_probs/init_probs.sum()).sum()

1.0

In [88]:
# sanity check for transition probabilities
(tran_probs/tran_probs.sum(axis=1)[:, np.newaxis]).sum(axis=1)

array([1., 1., 1., 1., 1., 1., 1., 1.])

In [95]:
final_tran_probs = tran_probs/tran_probs.sum(axis=1)[:, np.newaxis]
final_tran_probs

array([[3.36673990e-01, 1.95908309e-01, 2.39228107e-01, 2.28186162e-01,
        0.00000000e+00, 3.43276126e-06, 0.00000000e+00, 0.00000000e+00],
       [3.47248782e-01, 2.44139710e-01, 4.30627657e-02, 3.65536322e-01,
        0.00000000e+00, 1.24203267e-05, 0.00000000e+00, 0.00000000e+00],
       [2.54927379e-01, 2.02469356e-01, 2.44062515e-01, 2.98531777e-01,
        0.00000000e+00, 8.97388884e-06, 0.00000000e+00, 0.00000000e+00],
       [2.67780742e-01, 1.67633343e-01, 2.27956537e-01, 3.36624168e-01,
        0.00000000e+00, 5.21063363e-06, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.99077698e-01, 3.56594885e-01, 3.58744075e-01, 8.55833416e-02],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.89002316e-01, 3.56557190e-01, 2.92117022e-01, 1.62323473e-01],
       [6.34074381e-04, 8.41119077e-04, 1.30696964e-03, 5.11141593e-04,
        1.24278579e-01, 3.47272186e-01, 3.57223272e-01, 1.

In [96]:
final_init_probs = init_probs/init_probs.sum()
final_init_probs

array([0.29997011, 0.19703748, 0.19670899, 0.30027983, 0.00093958,
       0.00205016, 0.00206689, 0.00094697])

In [41]:
array_2d = np.array([[1, 2, 3],
                     [4, 5, 6],
                     [7, 8, 9]])

# Calculate the sum of elements along each row (axis=1)
row_sums = np.sum(array_2d, axis=1)

print("Original array:")
print(array_2d)
print("\nSum of elements per row:")
print(row_sums)

Original array:
[[1 2 3]
 [4 5 6]
 [7 8 9]]

Sum of elements per row:
[ 6 15 24]
